In [1]:
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc
import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('../input/training_set.csv',encoding='gbk')
test_df = pd.read_csv('../input/test_set.csv',encoding='gbk')
#vali_df = pd.read_csv('../input/validation_set.csv',encoding='gb18030')

In [3]:
#train_df = pd.concat([train_df, vali_df], axis=0)

In [4]:
train_df.shape

(24613, 2)

In [5]:
train_df.head()

,text,labels
0,飞扬 哥哥 晚上 好,positive
1,请说出 韩国 最佳 女子 组合 几个,positive
2,觉得 做 那种 人 最 幸福,positive
3,雨涵 这次 不错,positive
4,暗黑 破坏神 最 穿越 多久 没 更,negative


In [6]:
dit = {'positive':0,'negative':1}
train_df['labels'] = train_df['labels'].map(dit)

In [7]:
train_df.head()

,text,labels
0,飞扬 哥哥 晚上 好,0
1,请说出 韩国 最佳 女子 组合 几个,0
2,觉得 做 那种 人 最 幸福,0
3,雨涵 这次 不错,0
4,暗黑 破坏神 最 穿越 多久 没 更,1


In [8]:
#! -*- coding:utf-8 -*-
import re, os, json, codecs, gc
import numpy as np
import pandas as pd
from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam

maxlen = 512
# config_path = '../bert/chinese_wwm_ext_L-12_H-768_A-12/bert_config.json'
# # checkpoint_path = '/export/home/liuyuzhong/kaggle/bert/chinese_L-12_H-768_A-12/bert_model.ckpt'
# checkpoint_path = '../bert/chinese_wwm_ext_L-12_H-768_A-12/bert_model.ckpt'
# dict_path = '../bert/chinese_wwm_ext_L-12_H-768_A-12/vocab.txt'

config_path = '../bert/roeberta_zh_L-24_H-1024_A-16/bert_config_large.json'
# checkpoint_path = '/export/home/liuyuzhong/kaggle/bert/chinese_L-12_H-768_A-12/bert_model.ckpt'
checkpoint_path = '../bert/roeberta_zh_L-24_H-1024_A-16/roberta_zh_large_model.ckpt'
dict_path = '../bert/roeberta_zh_L-24_H-1024_A-16/vocab.txt'


token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

class data_generator:
    def __init__(self, data, batch_size=4, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            
            if self.shuffle:
                np.random.shuffle(idxs)
            
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y[:, 0, :]
                    [X1, X2, Y] = [], [], []

from keras.metrics import top_k_categorical_accuracy
def acc_top2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)
                    
def build_bert(nclass):
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

    for l in bert_model.layers:
        l.trainable = True

    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))

    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x)
    p = Dense(nclass, activation='softmax')(x)

    model = Model([x1_in, x2_in], p)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(1e-5),
                  metrics=['accuracy', acc_top2])
    print(model.summary())
    return model


In [9]:
from keras.utils import to_categorical

DATA_LIST = []
for data_row in train_df.iloc[:].itertuples():
    DATA_LIST.append((data_row.text, to_categorical(data_row.labels, 2)))
DATA_LIST = np.array(DATA_LIST)

DATA_LIST_TEST = []
for data_row in test_df.iloc[:].itertuples():
    DATA_LIST_TEST.append((data_row.text, to_categorical(0, 2)))
DATA_LIST_TEST = np.array(DATA_LIST_TEST)

In [10]:
def run_cv(nfold, data, data_label, data_test):
    kf = KFold(n_splits=nfold, shuffle=True, random_state=520).split(data)
    train_model_pred = np.zeros((len(data), 2))
    test_model_pred = np.zeros((len(data_test), 2))

    for i, (train_fold, test_fold) in enumerate(kf):
        X_train, X_valid, = data[train_fold, :], data[test_fold, :]
        
        model = build_bert(2)
        early_stopping = EarlyStopping(monitor='val_acc', patience=4)
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=1)
        checkpoint = ModelCheckpoint('./bert_dump/' + str(i) + '.hdf5', monitor='val_acc', 
                                         verbose=2, save_best_only=True, mode='max',save_weights_only=True)
        
        train_D = data_generator(X_train, shuffle=True)
        valid_D = data_generator(X_valid, shuffle=True)
        test_D = data_generator(data_test, shuffle=False)
        
        model.fit_generator(
            train_D.__iter__(),
            steps_per_epoch=len(train_D),
            epochs=15,
            validation_data=valid_D.__iter__(),
            validation_steps=len(valid_D),
            callbacks=[early_stopping, plateau, checkpoint],
        )
        
        # model.load_weights('./bert_dump/' + str(i) + '.hdf5')
        
        # return model
        train_model_pred[test_fold, :] =  model.predict_generator(valid_D.__iter__(), steps=len(valid_D),verbose=1)
        test_model_pred += model.predict_generator(test_D.__iter__(), steps=len(test_D),verbose=1)
        
        del model; gc.collect()
        K.clear_session()
        
        # break
        
    return train_model_pred, test_model_pred

In [11]:
train_model_pred, test_model_pred = run_cv(10, DATA_LIST, None, DATA_LIST_TEST)

W0922 22:51:17.167338 140464086943552 deprecation_wrapper.py:119] From /home/qishao/anaconda3/envs/qishao/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0922 22:51:17.181102 140464086943552 deprecation_wrapper.py:119] From /home/qishao/anaconda3/envs/qishao/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0922 22:51:17.229848 140464086943552 deprecation_wrapper.py:119] From /home/qishao/anaconda3/envs/qishao/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0922 22:51:17.230735 140464086943552 deprecation_wrapper.py:119] From /home/qishao/anaconda3/envs/qishao/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 1024)   324472832   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1024)         0           model_2[1][0]                    
__________

1250/1250 [==============================] - 43s 34ms/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 1024)   324472832   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1024)      


Epoch 00005: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.

Epoch 00005: val_acc did not improve from 0.98253
Epoch 6/15
5538/5538 [==============================] - 1749s 316ms/step - loss: 0.0031 - acc: 0.9994 - acc_top2: 1.0000 - val_loss: 0.1131 - val_acc: 0.9846 - val_acc_top2: 1.0000

Epoch 00006: val_acc improved from 0.98253 to 0.98456, saving model to ./bert_dump/4.hdf5
Epoch 7/15
5538/5538 [==============================] - 1749s 316ms/step - loss: 0.0023 - acc: 0.9994 - acc_top2: 1.0000 - val_loss: 0.0973 - val_acc: 0.9846 - val_acc_top2: 1.0000

Epoch 00007: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 00007: val_acc did not improve from 0.98456
Epoch 8/15
5538/5538 [==============================] - 1750s 316ms/step - loss: 7.6125e-04 - acc: 0.9998 - acc_top2: 1.0000 - val_loss: 0.1082 - val_acc: 0.9846 - val_acc_top2: 1.0000

Epoch 00008: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-07.

Epoch 00008: val_

Epoch 1/15
5538/5538 [==============================] - 1770s 320ms/step - loss: 0.1502 - acc: 0.9464 - acc_top2: 1.0000 - val_loss: 0.0941 - val_acc: 0.9683 - val_acc_top2: 1.0000

Epoch 00001: val_acc improved from -inf to 0.96831, saving model to ./bert_dump/6.hdf5
Epoch 2/15
5538/5538 [==============================] - 1740s 314ms/step - loss: 0.0596 - acc: 0.9809 - acc_top2: 1.0000 - val_loss: 0.0783 - val_acc: 0.9768 - val_acc_top2: 1.0000

Epoch 00002: val_acc improved from 0.96831 to 0.97684, saving model to ./bert_dump/6.hdf5
Epoch 3/15
5538/5538 [==============================] - 1739s 314ms/step - loss: 0.0363 - acc: 0.9886 - acc_top2: 1.0000 - val_loss: 0.0736 - val_acc: 0.9817 - val_acc_top2: 1.0000

Epoch 00003: val_acc improved from 0.97684 to 0.98171, saving model to ./bert_dump/6.hdf5
Epoch 4/15
5538/5538 [==============================] - 1740s 314ms/step - loss: 0.0275 - acc: 0.9924 - acc_top2: 1.0000 - val_loss: 0.0501 - val_acc: 0.9870 - val_acc_top2: 1.0000

Epoch

5538/5538 [==============================] - 1748s 316ms/step - loss: 2.6972e-07 - acc: 1.0000 - acc_top2: 1.0000 - val_loss: 0.1761 - val_acc: 0.9842 - val_acc_top2: 1.0000

Epoch 00015: ReduceLROnPlateau reducing learning rate to 9.765624753299562e-09.

Epoch 00015: val_acc did not improve from 0.98456
1250/1250 [==============================] - 43s 34ms/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, No

In [12]:
test_pred = [np.argmax(x) for x in test_model_pred]
test_df['labels'] = test_pred

In [13]:
test_df[['id', 'labels']].to_csv('baseline3.csv', index=None)

In [14]:
# np.save('bert_test_maxlen400_bs16_train.npy', train_model_pred)

In [15]:
dit = {0:'positive',1:'negative'}
test_df['labels'] = test_df['labels'].map(dit)

In [16]:
test_df[['id', 'labels']].to_csv('baseline_1.csv', index=None,header=None)